# MVD 4. cvičení

## 1. část - Načtení dat

Po rozbalení archive.zip uvidíte articles csv soubor. Tento soubor pochází z [Kaggle datasetů](https://www.kaggle.com/hsankesara/medium-articles) a obsahuje malé množství Medium článků k tématům ML, AI a data science. K úloze dnešního cvičení bude stačit využítí dat s názvy a obsahy článků (title a text).


### Příprava dat

Pro přípravu dat se použivá různá sekvence kroků. Je doporučeno na následující kroky vytvořit samostatnou funkci, aby bylo možné zpracovat i vyhledávaný výraz při testování. Dnešní cvičení by mělo obsahovat následující kroky:

1. Převést všechen text na lower case
2. Odstranění interpunkce a všech speciálních znaků (apostrof, ...)
3. Aplikace lemmatizátoru

Pozn.: Jedná se pouze o jednoduchý preprocessing, v praxi je často potřeba použití více kroků. Tato aplikace by měla například problém s čísly (desetinná čísla, čísla vyhledávaná slovně). 

Pro lemmatizaci použijte knihovnu spaCy.

In [1]:
# Instalace spaCy z Jupyter Notebooku
import sys
!{sys.executable} -m pip install spacy

# Stažení modelu pro angličtinu
!{sys.executable} -m spacy download en


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the full
pipeline package name 'en_core_web_sm' instead.
     --------------------------------------- 12.8/12.8 MB 11.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')



[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import spacy
lemmatizer = spacy.load('en_core_web_sm', disable=['parser', 'ner']) # NLTK
# Lemmatizace textu př.:  


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['parser', 'ner'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(


In [12]:
import csv
import os.path
def filetoLower(inf:str, outf:str, remove = False):
    if (not os.path.exists(outf)) or remove:
        with open(inf,"r",encoding="utf-8") as file, open(outf,"w",encoding="utf-8",newline='') as outFile:
            reader = csv.reader(file)
            writer = csv.writer(outFile)
            writer.writerow(next(reader))
            for line in reader:
                line[4] = line[4].lower()
                line[5] = line[5].lower()
                writer.writerow(line)
    else:
        print("file already exsists")
filetoLower("articles.csv","articlesLC.csv",True)

In [13]:
disabledChars = [',','.','\'','"','’','‘','(',')','“','”','?','/','\\','!','[',']','–','/','\\',':','-']
def filetoRemInter(inf:str, outf:str, remove = False):
    if (not os.path.exists(outf)) or remove:
        with open(inf,"r",encoding="utf-8") as file, open(outf,"w",encoding="utf-8",newline='') as outFile:
            reader = csv.reader(file)
            writer = csv.writer(outFile)
            writer.writerow(next(reader))
            for line in reader:
                for ch in disabledChars:
                    line[4] = line[4].replace(ch,' ')
                line[4] = " ".join(line[4].split())
                for ch in disabledChars:
                    line[5] = line[5].replace(ch,' ')
                line[5] = " ".join(line[5].split())
                writer.writerow(line)
    else:
        print("file already exsists")
filetoRemInter("articlesLC.csv","articlesLC_RI.csv",True)

In [16]:
def fileLematize(inf:str, outf:str, remove = False):
    if (not os.path.exists(outf)) or remove:
        with open(inf,"r",encoding="utf-8") as file, open(outf,"w",encoding="utf-8",newline='') as outFile:
            reader = csv.reader(file)
            writer = csv.writer(outFile)
            writer.writerow(next(reader))
            for line in reader:
                line[4] = " ".join([token.lemma_ for token in lemmatizer(line[4])])
                line[5] = " ".join([token.lemma_ for token in lemmatizer(line[5])])
                writer.writerow(line)
    else:
        print("file already exsists")
fileLematize("articlesLC_RI.csv","articlesLEMMA.csv",True)

## 2. část - Vytvoření invertovaného indexu

Před další prací s textem je potřeba vytvořit invertovaný index, který poté usnadní práci. Invertovaný index bude slovník, kde klíčem bude slovo a hodnotou bude list s id dokumentů (index), které dané slovo obsahují.

Pozn.: Je potřeba vytvořit dva invertované indexy - jeden pro title a druhý pro text.

In [3]:
import csv
def load(inf):
    cl = list()
    titleId = dict()
    textId = dict()
    count = 0
    with open(inf,"r",encoding="utf-8") as file:
        reader = csv.reader(file)
        next(reader)
        for i,line in enumerate(reader):
            cl.append(i)
            count = i
            for word in line[4].split():
                if not word in titleId:
                    titleId[word] = [i]
                else:
                    titleId[word].append(i)
            for word in line[5].split():
                if not word in textId:
                    textId[word] = [i]
                else:
                    textId[word].append(i)
    return titleId,textId,count
titleId,textId,count = load("articlesLEMMA.csv")
print(titleId['a'][:5],textId['a'][:100],count)


[38, 40, 41, 47, 64] [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 336


## 3. část - Implementace TF-IDF

Připravení funkce pro výpočet TF-IDF po příchodu dotazu. Funkce *tf_idf* by měla pracovat s dotazem, jedním invertovaným indexem a s danými dokumenty. Vrátit by měla list obsahující skóre pro každý dokument.

<br>
<center>
$
score(q,d) = TF\_IDF(q,d) = \sum\limits_{w \in q \cap d} c(w, q) c(w, d) log(\frac{M+1}{df(w)})
$
</center>

$q$ ... dotaz<br>
$d$ ... dokument<br>
$c(w, q)$ ... kolikrát je slovo *w* v dotazu *q*<br>
$M$ ... celkový počet dokumentů<br>
$df(w)$ ... počet dokumentů, ve kterých se nachází slovo *w*

In [22]:
import math
def cq(w:str,qd:dict):
    if w in qd:
        return qd[w]
    else:
        return 0

def cd(w:str, docID:dict ,docIndex:int):
    if w in docID:
        return docID[w].count(docIndex)
    else:
        return 0
def df(w:str, docID:dict):
    if w in docID:
        ln = len(set(docID[w]))
        if(ln>0):
            return len(set(docID[w]))
        else:
            return 1

    else:
        return 1

def scoreDoc(q:str,d:int,doc:dict):
    qd = dict()
    for word in q.split(' '):
        if word in qd:
            qd[word]+=1
        else:
            qd[word]=1
    sums = 0
    for word in q.split(' '):
        sums+= cq(word,qd)* cd(word,doc,d) * math.log((count+1)/(df(q,doc)))
    return sums
def scoreText(q:str,d:int):
    return scoreDoc(q,d,textId)     

def scoreTitle(q:str,d:int):
    return scoreDoc(q,d,titleId)     

print(scoreText("a",0))

0.0


## 4. část - Použití a testování TF-IDF

Nyní lze získat skóre pro titulky nebo text. Následujícím krokem je sjednocení výsledného skóre pro ohodnocení celého dokumentu. V případě dvou hodnot si vystačíme s parametrem $\alpha$, který nám určuje jakou váhu má titulek a jakou samotný text dokumentu. <br>

<center>
$
score(q,d) = \alpha \; TF\_IDF\_title(q,d) + (1-\alpha) \; TF\_IDF\_text(q,d)
$
</center>

Při nastavení parametru $\alpha$ na hodnotu 0.7 a vyhledávání dotazu "coursera vs udacity machine learning" by výsledky měly vypadat následovně:

![output](sample_output.png)

In [30]:
def scoreDocTitle(q:str,d:int,alpha:float = 0.7):
    return alpha * scoreTitle(q,d)+ (1-alpha) * scoreText(q,alpha)
tts = list()
text = "coursera vs udacity machine learning"
for i in range(count):
    tts.append((i,scoreDocTitle(text,i)))
tts.sort(key = lambda x: x[1],reverse=True)
titles = list()
texts = list()
with open("articles.csv","r",encoding="utf-8") as file:
    reader = csv.reader(file)
    next(reader)
    for line in reader:
        titles.append(line[4])
        texts.append(line[5])

for i in range(10):
    print(tts[i][0],titles[tts[i][0]][:40],'... :',texts[tts[i][0]][:40],'... :',tts[i][1])



276 Coursera vs Udacity for Machine Learning ... : 2018 is an exciting time for students of ... : 20.370290256233265
14 Machine Learning is Fun! Part 3: Deep Le ... : Update: This article is part of a series ... : 12.222174153739957
15 Machine Learning is Fun! Part 4: Modern  ... : Update: This article is part of a series ... : 12.222174153739957
56 Machine Learning is Fun! Part 3: Deep Le ... : Update: This article is part of a series ... : 12.222174153739957
57 Machine Learning is Fun! Part 4: Modern  ... : Update: This article is part of a series ... : 12.222174153739957
60 Machine Learning is Fun Part 6: How to d ... : Update: This article is part of a series ... : 12.222174153739957
61 Machine Learning is Fun Part 5: Language ... : Update: This article is part of a series ... : 12.222174153739957
138 Machine Learning is Fun Part 6: How to d ... : Update: This article is part of a series ... : 12.222174153739957
139 Machine Learning is Fun Part 5: Language ... : Update: This articl